In [17]:
import awkward as ak 
import json

from binary_decoders import etroc
import importlib

In [ ]:
importlib.reload(etroc)

my_output = etroc.converter(
    ["unit_test/input_data/run_6000/output_run_6000_rb0.dat", 
     "unit_test/input_data/run_6000/output_run_6000_rb1.dat", 
     "unit_test/input_data/run_6000/output_run_6000_rb2.dat"],
    skip_trigger_check=True
)

In [ ]:
print(my_output['nhits'].to_list())

In [ ]:
print(38<<2)
print(36<<2)
print(37<<2)

In [1]:
import logging
from functools import wraps
logging.basicConfig(level=logging.DEBUG)

logger = logging.getLogger("conditional_counter")

class DuplicateFilter(logging.Filter):
    def __init__(self, logger):
        super().__init__()
        self.logged_messages = set()
        self.logger = logger
        self.found_ifs = []
    def filter(self, record):
        if record.msg not in self.logged_messages:
            if_number = getattr(record, 'if_number')
            print(f"IF NUMBER: {if_number}")
            self.found_ifs.append(if_number)
            print(f"FOUND IFS: {self.found_ifs}")
            self.logged_messages.add(record.msg)

            
            return True  
        return False

    def __call__(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            logger.addFilter(self)
            logger.setFormatter(logging.Formatter(' %(name)s - %(levelname)s - %(message)s - %(if_number)%'))
            try:
                return func(*args, **kwargs)
            finally:
                logger.removeFilter(self)
        return wrapper


@DuplicateFilter(logger)
def log_messages(a):
    for _ in range(5):
        logger.info("This will only show up once!", extra={'if_number': 1})
        #logger.error(ValueError("So will this!"))
        if a==2:
            logger.info("fart", extra={'if_number': 2})

log_messages(1)

log_messages(2)

INFO:conditional_counter:This will only show up once!
INFO:conditional_counter:fart


IF NUMBER: 1
FOUND IFS: [1]
IF NUMBER: 2
FOUND IFS: [1, 2]
